Generate

In [ ]:
import sys
import os
import torch
from optimizer import Optimizer
from config import Config
import glob
sys.path.insert(0,'/content/NextFace') # Verify your path

config = Config()
config.fillFromDicFile('./optimConfig.ini')
# config.device = 'cuda' # torch not compiled with CUDA
config.path = './baselMorphableModel/' # Verify your path

# Directory path containing all images
imageFolderPath = './input/10_famous_people/'

outputDir = './output/'
#setup
# Extract the folder name (e.g., '10_famous_people')
folder_name = os.path.basename(imageFolderPath.strip('/'))

outputDir = './output/' + folder_name + '/'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)  # Create the output directory if it doesn't exist
    
torch.cuda.set_device(0)

# Loop through all JPG images in the specified folder
for imagePath in glob.glob(imageFolderPath + '*.jpg'):
    # Determine output directory based on image name
    image_name = os.path.basename(imagePath.strip('/'))
    outputImageDir = outputDir + image_name
    optimizer = Optimizer(outputImageDir, config)
    
    # Run the optimization for the current image
    optimizer.run(imagePath, doStep1=True, doStep2=True, doStep3=False, renderer="vertex")
    # Based on the vertex_based optimization, try to get to the same result with the Mitsuba one
    optimizer.run(outputImageDir+'/results/ref.png', checkpoint=outputImageDir+'/checkpoints/stage1_output.pickle', doStep1=False, doStep2=True, doStep3=False, renderer="mitsuba")

Results

In [ ]:
#display results
# Directory containing the images
dir_path = outputDir + '/debug/Baseline/mitsuba/'

# Get a list of all the .png images in the directory
image_files = glob.glob(os.path.join(dir_path, "*.png"))

# Display each image in turn
for image_file in image_files:
    print(f"Displaying image: {os.path.basename(image_file)}")
    display(Image(filename=image_file))


Mesh

In [ ]:
# # display meshes in polyscope
# import polyscope as ps
# import trimesh
# import glob
# import math

# display_meshes = ["vertex_based_step2_iter0.obj", "vertex_based_step2_iter100.obj","vertex_based_step2_iter200.obj","vertex_based_step2_iter300.obj","vertex_based_step2_iter400.obj"]  # replace this with your mesh names


# # Get a list of all .obj files in the directory
# obj_files = glob.glob(os.path.join(outputDir + '/debug/mesh/', "*.obj"))

# # Filter obj_files to only include the ones in display_meshes
# obj_files = [file for file in obj_files if os.path.basename(file) in display_meshes]

# # Define a folder where the screenshots will be saved
# screenshot_folder = outputDir+ "/debug/screenshot_folder"  # replace with your desired folder

# # Check if the folder exists, if not create it
# os.makedirs(screenshot_folder, exist_ok=True)

# # Initialize polyscope
# ps.init()
# # enable auto centering and scaling
# # ps.set_autocenter_structures(True)
# # ps.set_autoscale_structures(True)
# ps.set_up_dir("neg_y_up")
# # ps.set_bounding_box(-1.0,2.0)
# # ps.set_SSAA_factor(4)
# # ps.set_automatically_compute_scene_extents(True)
# # ps.set_autoscale_structures(True)
# # ps.set_autocenter_structures(True)
# # ps.set_ground_plane_height_factor(1.0,True)
# ps.set_ground_plane_mode("none")

# # # Show the mesh
# # ps.show()
# # Load and register each mesh to polyscope
# for idx, obj_file in enumerate(obj_files):
#     # mesh_name = os.path.basename(obj_file)  # get the name of the mesh
#     mesh_name = os.path.basename(obj_file)  # get the name of the mesh

#     # Load the mesh from an obj file using trimesh
#     mesh = trimesh.load_mesh(obj_file)
#     # Register the mesh to polyscope
#     ps_mesh = ps.register_surface_mesh(mesh_name, mesh.vertices, mesh.faces)
#     # Reset the transformation on the mesh
#     ps_mesh.reset_transform()
#     # Set a color for the mesh
#     color = torch.tensor([0.5, 0.5, 0.5]).numpy()  # set a fixed color
#     ps_mesh.set_color(color)
#     # Reset the view
#     ps.reset_camera_to_home_view()
#     # Show the mesh
#     ps.show()

#     # Take a screenshot and save it to the specified folder
#     screenshot_name = os.path.join(screenshot_folder, f"{mesh_name}_screenshot.png")
#     ps.screenshot(screenshot_name)

#     # Remove the mesh from polyscope to prepare for the next one
#     ps.remove_all_structures()
# #display results
# # Directory containing the images
# dir_path = outputDir + '/debug/screenshot_folder/'

# # Get a list of all the .png images in the directory
# image_files = glob.glob(os.path.join(dir_path, "*.png"))

# # Display each image in turn
# for image_file in image_files:
#     print(f"Displaying image: {os.path.basename(image_file)}")
#     display(Image(filename=image_file))